In [ ]:
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes

In [2]:
! pip install transformers datasets evaluate


   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.1 MB 8.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/10.1 MB 9.9 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.1 MB 9.3 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.1 MB 9.8 MB/s eta 0:00:01
   -------------------------------------- - 9.7/10.1 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------------------------- ---- 2.1/2.4 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 8.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (TrainingArguments,
                          pipeline,
                          DistilBertForQuestionAnswering,
                          DistilBertTokenizer,
                          DistilBertTokenizerFast,
                          Trainer,
                          logging)
from datasets import Dataset, load_dataset
import evaluate
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
df_train = pd.read_csv("/data/train.csv") #Transformer\Week 1\processed_data
df_test = pd.read_csv("/data/test.csv")

In [ ]:
import re
def clean_answer(answer):
    # Remove annotations like <<>> and final '#### ...'
    answer = re.sub(r'<<.*?>>', '', answer)
    answer = re.sub(r'####.*', '', answer)
    return answer.strip()

df_train['clean_answer'] = df_train['answer'].apply(clean_answer)
df_test['clean_answer'] = df_test['answer'].apply(clean_answer)

# Split answers into stepwise reasoning
def split_steps(answer):
    return answer.split('\n')

df_train['steps'] = df_train['clean_answer'].apply(split_steps)
df_test['steps'] = df_test['clean_answer'].apply(split_steps)

# 3. Prepare dataset for BERT input
def create_input_output_pairs(row):
    return {
        'question': row['question'],
        'answer': row['clean_answer']
    }

data_pairs_train = df_train.apply(create_input_output_pairs, axis=1).tolist()
data_pairs_test = df_test.apply(create_input_output_pairs, axis=1).tolist()

# Convert to Hugging Face Datasets
dataset_train = Dataset.from_list(data_pairs_train)
dataset_test = Dataset.from_list(data_pairs_test)

print(dataset_train[0])
print(dataset_test[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = 24 clips in May.\nNatalia sold 48+24 = 72 clips altogether in April and May.'}
{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = 9 duck eggs a day.\nShe makes 9 * 2 = $18 every day at the farmer’s market.'}


In [ ]:
def tokenize_function(example):
    # Kết hợp câu hỏi và câu trả lời để tạo ngữ cảnh đầy đủ
    context = example['question'].strip() + " " + example['answer'].strip()

    # Token hóa ngữ cảnh
    tokenized = tokenizer(
        context,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_offsets_mapping=True
    )

    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']
    answer = example['answer'].strip()

    # Tìm vị trí bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
    answer_start = context.lower().find(answer.lower())
    if answer_start == -1:  # Nếu không tìm thấy câu trả lời
        logging.warning(f"Answer not found in context! Skipping example.")
        return None

    answer_end = answer_start + len(answer)

    # Ánh xạ vị trí ký tự sang vị trí token
    offset_mapping = tokenized['offset_mapping']
    start_positions, end_positions = None, None

    for i, (start, end) in enumerate(offset_mapping):
        if start <= answer_start < end:
            start_positions = i
        if start < answer_end <= end:
            end_positions = i
            break

    # Kiểm tra tính hợp lệ của các vị trí token
    if start_positions is None or end_positions is None:
        logging.warning(f"Token mapping failed for example. Skipping example.")
        return None

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'start_positions': start_positions,
        'end_positions': end_positions
    }


# Tokenize datasets
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=False)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/14946 [00:00<?, ? examples/s]

Map:   0%|          | 0/2638 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# 4. Define and train BERT model
model = DistilBertForQuestionAnswering.from_pretrained('bert-base-uncased')

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=500,  # Match the evaluation frequency to save steps
    learning_rate=2e-5,
    warmup_steps=500,  # Warmup for learning rate scheduling
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    num_train_epochs=5,  # Increased epochs for better convergence
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    load_best_model_at_end=True  # Load the best model at the end of training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)
# Train the model
print("\nStarting training...")
trainer.train()

# Save the model
model.save_pretrained("bert_gsm8k_qa")
tokenizer.save_pretrained("bert_gsm8k_qa")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'qa_outputs.bias', 'qa_outputs.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.sa_lay


Starting training...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="/content/drive/MyDrive/bert_gsm8k_qa")

# Bài toán từ GSM8K
context = """
Alice has 3 apples, Bob has 5 apples, and Charlie has 2 apples. They decide to pool their apples together
and share them equally among themselves. How many apples will each person get?
"""
question = "How many apples will each person get?"

# Thực hiện suy luận
result = qa_pipeline(question=question, context=context)

# In kết quả
print(f"Question: {question}")
print(f"Answer: {result['answer']}")
print(f"Score: {result['score']:.2f}")

Device set to use cpu


Question: How many apples will each person get?
Answer: ?
Score: 0.00
